In [68]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
df = pd.read_csv(r"data\train.csv", sep = ',')
df

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5237980 entries, 0 to 5237979
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   stock_id                 int64  
 1   date_id                  int64  
 2   seconds_in_bucket        int64  
 3   imbalance_size           float64
 4   imbalance_buy_sell_flag  int64  
 5   reference_price          float64
 6   matched_size             float64
 7   far_price                float64
 8   near_price               float64
 9   bid_price                float64
 10  bid_size                 float64
 11  ask_price                float64
 12  ask_size                 float64
 13  wap                      float64
 14  target                   float64
 15  time_id                  int64  
 16  row_id                   object 
dtypes: float64(11), int64(5), object(1)
memory usage: 679.4+ MB


In [5]:
#df_prova = df.loc[df['stock_id'] == 0]
#df_prova = df_prova.loc[df_prova['date_id'] == 0]
#df_prova = df.copy(deep = True)

## 1. Data Analysis

In [ ]:
df_prova_notna = df_prova.loc[\
    pd.notna(df_prova['far_price']) & pd.notna(df_prova['near_price']) & pd.notna(df_prova['reference_price'])]

In [ ]:
df_prova_notna

In [1]:
plt.figure(figsize=(12,12))
cor = df_prova_notna.corr(method='pearson')
sns.heatmap(cor, annot=True)
plt.show()

NameError: name 'plt' is not defined

In [ ]:
print(len(df))
df.isnull().sum(axis=0)

In [ ]:
df[df['stock_id'] == 158]

In [ ]:
df_prova_notna = df_prova.loc[\
    pd.notna(df_prova['far_price']) & pd.notna(df_prova['near_price']) & pd.notna(df_prova['reference_price'])]

## 2. Data Preprocessing

In [6]:
df_5_9_min = df.loc[df['seconds_in_bucket'] >= 300]
df_5_9_min.drop(columns = ['row_id'], inplace = True)

In [7]:
print(len(df_5_9_min))
df_5_9_min.isnull().sum(axis=0)

2380900


stock_id                       0
date_id                        0
seconds_in_bucket              0
imbalance_size               100
imbalance_buy_sell_flag        0
reference_price              100
matched_size                 100
far_price                  37262
near_price                   100
bid_price                    100
bid_size                       0
ask_price                    100
ask_size                       0
wap                          100
target                        26
time_id                        0
row_id                         0
dtype: int64

In [8]:
df_5_9_min.loc[pd.isna(df_5_9_min['far_price'])]

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
5763,33,0,300,3092376.98,-1,0.997339,8978868.85,NaN,0.958708,0.996829,16547.96,0.997922,18345.94,0.997348,6.040335,30,0_300_33
5803,74,0,300,10011877.82,-1,0.998111,25349282.16,NaN,0.939632,0.998111,698.76,0.999197,6295.68,0.998220,-12.279749,30,0_300_74
5816,89,0,300,3752394.06,-1,0.998701,8351193.72,NaN,0.951207,0.997631,30906.00,0.999030,3034.25,0.998905,-12.260079,30,0_300_89
5818,91,0,300,9722538.89,1,1.001756,19001042.87,NaN,1.030012,1.001926,10030.68,1.002605,295.22,1.002586,-17.600060,30,0_300_91
5954,33,0,310,2974526.41,-1,0.996829,9096719.42,NaN,0.962498,0.996829,52242.32,0.997047,70310.06,0.996922,8.230209,31,0_310_33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5236124,144,480,450,9083704.04,-1,0.998723,27319450.26,NaN,0.900102,0.998723,8798.61,0.998845,90052.80,0.998734,2.839565,26445,480_450_144
5236151,171,480,450,7389482.22,-1,0.999281,22202291.97,NaN,0.950294,0.999371,13216.00,0.999460,22514.01,0.999404,0.070333,26445,480_450_171
5236167,187,480,450,8083304.67,1,1.000180,29493925.41,NaN,1.038808,1.000026,30150.72,1.000180,11178.28,1.000138,-4.169941,26445,480_450_187
5236240,60,480,460,29976877.98,-1,0.990839,75750325.85,NaN,0.949708,0.990771,44953.10,0.990839,40315.56,0.990807,-2.679825,26446,480_460_60


In [9]:
df_5_9_min.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2380900 entries, 5730 to 5237979
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   stock_id                 int64  
 1   date_id                  int64  
 2   seconds_in_bucket        int64  
 3   imbalance_size           float64
 4   imbalance_buy_sell_flag  int64  
 5   reference_price          float64
 6   matched_size             float64
 7   far_price                float64
 8   near_price               float64
 9   bid_price                float64
 10  bid_size                 float64
 11  ask_price                float64
 12  ask_size                 float64
 13  wap                      float64
 14  target                   float64
 15  time_id                  int64  
 16  row_id                   object 
dtypes: float64(11), int64(5), object(1)
memory usage: 327.0+ MB


In [13]:
df_5_9_filled = df_5_9_min.groupby(['stock_id', 'date_id'], group_keys=False).apply(lambda s: s.interpolate())

In [55]:
df_5_9_filled.loc[pd.isna(df_5_9_min['far_price'])]

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
5763,33.0,0.0,300.0,3092376.98,-1.0,0.997339,8978868.85,0.922409,0.958708,0.996829,16547.96,0.997922,18345.94,0.997348,6.040335,30.0
5803,74.0,0.0,300.0,10011877.82,-1.0,0.998111,25349282.16,0.899166,0.939632,0.998111,698.76,0.999197,6295.68,0.998220,-12.279749,30.0
5816,89.0,0.0,300.0,3752394.06,-1.0,0.998701,8351193.72,0.000412,0.951207,0.997631,30906.00,0.999030,3034.25,0.998905,-12.260079,30.0
5818,91.0,0.0,300.0,9722538.89,1.0,1.001756,19001042.87,1.050049,1.030012,1.001926,10030.68,1.002605,295.22,1.002586,-17.600060,30.0
5954,33.0,0.0,310.0,2974526.41,-1.0,0.996829,9096719.42,0.922409,0.962498,0.996829,52242.32,0.997047,70310.06,0.996922,8.230209,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5236124,144.0,480.0,450.0,9083704.04,-1.0,0.998723,27319450.26,0.959129,0.900102,0.998723,8798.61,0.998845,90052.80,0.998734,2.839565,26445.0
5236151,171.0,480.0,450.0,7389482.22,-1.0,0.999281,22202291.97,0.994195,0.950294,0.999371,13216.00,0.999460,22514.01,0.999404,0.070333,26445.0
5236167,187.0,480.0,450.0,8083304.67,1.0,1.000180,29493925.41,1.037577,1.038808,1.000026,30150.72,1.000180,11178.28,1.000138,-4.169941,26445.0
5236240,60.0,480.0,460.0,29976877.98,-1.0,0.990839,75750325.85,0.971640,0.949708,0.990771,44953.10,0.990839,40315.56,0.990807,-2.679825,26446.0


In [30]:
df_5_9_filled = df_5_9_min.groupby(['stock_id', 'date_id'], group_keys=False).apply(lambda s: (s.ffill().astype(float)+s.bfill().astype(float))/2)

In [31]:
df_5_9_filled

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
5730,0.0,0.0,300.0,0.00,0.0,1.000241,26670118.88,1.000241,1.000241,1.000026,19319.31,1.000241,16149.55,1.000143,-10.039806,30.0,3000.0
5731,1.0,0.0,300.0,242332.96,-1.0,1.000073,3242054.27,0.981974,0.994490,0.999544,43205.40,1.000308,2042.76,1.000273,-1.329780,30.0,3001.0
5732,2.0,0.0,300.0,0.00,0.0,1.000193,4671376.00,1.000193,1.000193,0.999035,18971.00,1.001036,59688.26,0.999518,14.009476,30.0,3002.0
5733,3.0,0.0,300.0,2914730.16,1.0,0.999870,41057776.66,1.003870,1.002279,0.999827,25569.50,1.000042,37897.50,0.999914,-4.339814,30.0,3003.0
5734,4.0,0.0,300.0,3396923.02,1.0,0.998496,38356174.88,1.007894,1.007894,0.998357,14446.00,0.998703,2601.18,0.998650,10.850430,30.0,3004.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195.0,480.0,540.0,2440722.89,-1.0,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454.0,480540195.0
5237976,196.0,480.0,540.0,349510.47,-1.0,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454.0,480540196.0
5237977,197.0,480.0,540.0,0.00,0.0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454.0,480540197.0
5237978,198.0,480.0,540.0,1000898.84,1.0,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454.0,480540198.0


In [70]:
def fill_NaN(s: pd.Series) -> pd.Series:
    s_bfill = s.bfill()
    s_ffill = s.ffill()
    s_bfill = s_bfill.fillna(s_ffill)
    s_ffill = s_ffill.fillna(s_bfill)
    
    return (s_bfill + s_ffill)/2

In [88]:
df_5_9_filled = pd.DataFrame()
for colonna in tqdm(df_5_9_min.columns[2:]):
    df_5_9_filled[colonna] = df_5_9_min[['stock_id', 'date_id', colonna]].groupby(['stock_id', 'date_id'], group_keys=True).apply(lambda s: fill_NaN(s[colonna]))

#df_5_9_filled = df_5_9_min.groupby(['stock_id', 'date_id'], group_keys=False).apply(lambda s: fill_NaN(s))

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [12:33<00:00, 53.83s/it]


In [ ]:
df_5_9_filled.to_csv(r'data\df_5_9_filled_V1.csv', sep=',')

In [89]:
df_5_9_filled

seconds_in_bucket  imbalance_size  \
stock_id date_id                                              
0        0       5730                 300.0            0.00   
                 5921                 310.0            0.00   
                 6112                 320.0            0.00   
                 6303                 330.0            0.00   
                 6494                 340.0            0.00   
...                                     ...             ...   
199      480     5237179              500.0      2661783.46   
                 5237379              510.0      2658917.61   
                 5237579              520.0      1352558.88   
                 5237779              530.0      1352558.88   
                 5237979              540.0      1884285.71   

                          imbalance_buy_sell_flag  reference_price  \
stock_id date_id                                                     
0        0       5730                         0.0         1.000241   
                 5921                         0.0         0.999919   
                 6112                         0.0         0.999919   
                 6303                         0.0         0.999812   
                 6494                         0.0         0.999491   
...                                           ...              ...   
199      480     5237179                     -1.0         1.002764   
                 5237379                     -1.0         1.002764   
                 5237579                     -1.0         1.001812   
                 5237779                     -1.0         1.001812   
                 5237979                     -1.0         1.002129   

                          matched_size  far_price  near_price  bid_price  \
stock_id date_id                                                           
0        0       5730      26670118.88   1.000241    1.000241   1.000026   
                 5921      26670118.88   0.999919    0.999919   0.999812   
                 6112      26670118.88   0.999919    0.999919   0.999705   
                 6303      26670118.88   0.999812    0.999812   0.999705   
                 6494      26670118.88   0.999491    0.999491   0.999169   
...                                ...        ...         ...        ...   
199      480     5237179   23969215.23   1.000859    1.001494   1.002764   
                 5237379   23972081.09   1.000859    1.001494   1.002764   
                 5237579   23978379.68   1.000859    1.001494   1.001812   
                 5237779   23978379.68   1.000859    1.001494   1.001812   
                 5237979   24073677.32   1.000859    1.001494   1.002129   

                           bid_size  ask_price   ask_size       wap  \
stock_id date_id                                                      
0        0       5730      19319.31   1.000241   16149.55  1.000143   
                 5921      22767.64   0.999919   18664.00  0.999871   
                 6112      49262.40   0.999919   37607.96  0.999826   
                 6303      27990.00   0.999812   14276.43  0.999776   
                 6494      18743.25   0.999383   40012.83  0.999237   
...                             ...        ...        ...       ...   
199      480     5237179  122246.18   1.003082  425296.17  1.002835   
                 5237379  677012.04   1.003082  347268.87  1.002974   
                 5237579  225361.65   1.002129  194630.52  1.001982   
                 5237779  285559.05   1.002129  214513.32  1.001993   
                 5237979  250081.44   1.002447  300167.56  1.002274   

                             target  time_id  
stock_id date_id                              
0        0       5730    -10.039806     30.0  
                 5921     -6.039739     31.0  
                 6112     -5.440116     32.0  
                 6303     -7.439852     33.0  
                 6494     -2.700090     34.0  
...                             ...      ...  
199   

In [85]:
df_merge_prova = pd.merge(df_5_9_min[['stock_id', 'date_id']], df_5_9_filled,
                          left_index=True,
                          right_index=True)

In [87]:
df_merge_prova.loc[pd.isna(df_merge_prova['far_price'])]

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
227377,78,21,300.0,2180217.61,1.0,1.014278,2648727.34,NaN,1.024576,1.013820,4651.50,1.015651,19438.44,1.014174,2.750158,1185.0
227569,78,21,310.0,1872465.88,1.0,1.015880,2775664.55,NaN,1.022059,1.015193,37306.76,1.016109,178266.00,1.015352,-27.819872,1186.0
227761,78,21,320.0,1859357.04,1.0,1.016566,2788773.39,NaN,1.024347,1.015880,16424.30,1.016566,189895.50,1.015934,-32.629967,1187.0
227953,78,21,330.0,1822215.35,1.0,1.015880,2825915.08,NaN,1.022974,1.014278,17816.64,1.016109,57409.20,1.014712,-15.550256,1188.0
228145,78,21,340.0,1822215.35,1.0,1.016338,2825915.08,NaN,1.025263,1.015193,6299.12,1.016566,27362.72,1.015450,-35.300255,1189.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5204123,143,477,500.0,255151.04,1.0,1.004842,255515.84,NaN,1.004842,0.998483,283584.39,1.004842,580601.02,1.000579,-8.919835,26285.0
5204323,143,477,510.0,255151.04,1.0,1.004842,255515.84,NaN,1.004842,0.998483,282171.39,1.004842,579806.28,1.000573,-5.769730,26286.0
5204523,143,477,520.0,255151.04,1.0,1.004842,255515.84,NaN,1.004842,0.998483,288765.39,1.004842,578066.70,1.000610,-7.150173,26287.0
5204723,143,477,530.0,255151.04,1.0,1.004842,255515.84,NaN,1.004842,0.998483,287363.38,1.004842,578066.70,1.000603,-9.520054,26288.0


In [90]:
df_5_9_filled.loc[pd.isna(df_5_9_filled['far_price'])]

seconds_in_bucket  imbalance_size  \
stock_id date_id                                              
19       438     4770999              300.0             NaN   
                 4771199              310.0             NaN   
                 4771399              320.0             NaN   
                 4771599              330.0             NaN   
                 4771799              340.0             NaN   
...                                     ...             ...   
158      388     4225138              500.0             NaN   
                 4225338              510.0             NaN   
                 4225538              520.0             NaN   
                 4225738              530.0             NaN   
                 4225938              540.0             NaN   

                          imbalance_buy_sell_flag  reference_price  \
stock_id date_id                                                     
19       438     4770999                     -1.0              NaN   
                 4771199                     -1.0              NaN   
                 4771399                     -1.0              NaN   
                 4771599                     -1.0              NaN   
                 4771799                     -1.0              NaN   
...                                           ...              ...   
158      388     4225138                      0.0              NaN   
                 4225338                      0.0              NaN   
                 4225538                      0.0              NaN   
                 4225738                      0.0              NaN   
                 4225938                      0.0              NaN   

                          matched_size  far_price  near_price  bid_price  \
stock_id date_id                                                           
19       438     4770999           NaN        NaN         NaN        NaN   
                 4771199           NaN        NaN         NaN        NaN   
                 4771399           NaN        NaN         NaN        NaN   
                 4771599           NaN        NaN         NaN        NaN   
                 4771799           NaN        NaN         NaN        NaN   
...                                ...        ...         ...        ...   
158      388     4225138           NaN        NaN         NaN        NaN   
                 4225338           NaN        NaN         NaN        NaN   
                 4225538           NaN        NaN         NaN        NaN   
                 4225738           NaN        NaN         NaN        NaN   
                 4225938           NaN        NaN         NaN        NaN   

                          bid_size  ask_price  ask_size  wap    target  \
stock_id date_id                                                         
19       438     4770999  85760.64        NaN   7551.74  NaN -6.819964   
                 4771199  12308.24        NaN   5366.79  NaN -2.400279   
                 4771399   1786.68        NaN   3975.00  NaN  3.130436   
                 4771599   1786.68        NaN   5167.24  NaN  4.539490   
                 4771799   1985.20        NaN   4968.00  NaN  8.080006   
...                            ...        ...       ...  ...       ...   
158      388     4225138      0.00        NaN      0.00  NaN       NaN   
                 4225338      0.00        NaN      0.00  NaN       NaN   
                 4225538      0.00        NaN      0.00  NaN       NaN   
                 4225738      0.00        NaN      0.00  NaN       NaN   
                 4225938      0.00        NaN      0.00  NaN       NaN   

                          time_id  
stock_id date_id                   
19       438     4770999  24120.0  
                 4771199  24121.0  
                 4771399  24122.0  
                 4771599  24123.0  
                 4771799  24124.0  
...                           ...  
158      388     4225138  21390.0  
                 42

In [94]:
df_5_9_filled.loc[19].loc[438]

,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
4770999,300.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,85760.64,NaN,7551.74,NaN,-6.819964,24120.0
4771199,310.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,12308.24,NaN,5366.79,NaN,-2.400279,24121.0
4771399,320.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1786.68,NaN,3975.00,NaN,3.130436,24122.0
4771599,330.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1786.68,NaN,5167.24,NaN,4.539490,24123.0
4771799,340.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1985.20,NaN,4968.00,NaN,8.080006,24124.0
4771999,350.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,2977.80,NaN,4768.80,NaN,7.590056,24125.0
4772199,360.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,2977.80,NaN,5166.20,NaN,10.360479,24126.0
4772399,370.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,28786.85,NaN,198.67,NaN,6.459951,24127.0
4772599,380.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,79018.92,NaN,5165.42,NaN,4.429817,24128.0
4772799,390.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,88350.30,NaN,5165.42,NaN,0.050068,24129.0


In [77]:
df_5_9_min[['stock_id', 'date_id', colonna]]

,stock_id,date_id,seconds_in_bucket
5730,0,0,300
5731,1,0,300
5732,2,0,300
5733,3,0,300
5734,4,0,300
...,...,...,...
5237975,195,480,540
5237976,196,480,540
5237977,197,480,540
5237978,198,480,540


In [79]:
print(len(df_5_9_filled))
df_5_9_filled.isnull().sum(axis=0)

2380900


seconds_in_bucket            0
imbalance_size             100
imbalance_buy_sell_flag      0
reference_price            100
matched_size               100
far_price                  600
near_price                 100
bid_price                  100
bid_size                     0
ask_price                  100
ask_size                     0
wap                        100
target                      25
time_id                      0
dtype: int64

In [ ]:
df_5_9_filled = (df_5_9_min.ffill()+df_5_9_min.bfill())/2
groupby().apply(lambda (df.ffill()+df.bfill())/2)

## 3. Feature Selection

## 4. Feature Engineering

## Data Modeling

Training Set

    1: 2
    2: 3
    3: 4
    4: 5
    5: 6
    6: 7
    7: 8

Validation Set

    8: 9

Test Set

    9: 10